In [16]:
import numpy as np

**1**. (25 points)

The following iterative sequence is defined for the set of positive integers:

- n → n/2 (n is even)
- n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:

13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1

It can be seen that this sequence (starting at 13 and finishing at 1) contains 10 terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at 1.

Which starting number, under one million, produces the longest chain?

NOTE: Once the chain starts the terms are allowed to go above one million.

In [12]:
# Initialize result variables
largest_start_num = 1 # Starting number for sequence with longest chain
longest_chain_len = 1 # Number of terms in longest sequence

# Iterate through all integers less than one million
for num in range(1, 1000000):
    # Reset chain length, and preserve original value of "num" before it is altered
    chain_len = 0
    storage_value = num
    # Perform sequence rules until 1 is reached
    while num > 1:
        if num % 2 == 0:
            num /= 2
        else:
            num = 3*num + 1
        chain_len += 1
    # If this sequence is the longest, update the records
    if chain_len > longest_chain_len:
        longest_chain_len = chain_len
        largest_start_num = storage_value

# Display result
print(largest_start_num)

837799


In [ ]:
### Consider making code more efficient using a dictionary to store results that we've already seen

**2** (25 points)


- Perform the median polish to calculate just the *residuals* for this [example](https://mgimond.github.io/ES218/Week11a.html) in Python. 
- Use the matrix `xs` provided
- Display the final result after 3 iterations to 1 decimal place and check if it agrees with 

![img](https://mgimond.github.io/ES218/img/twoway_09.jpg)

In [13]:
xs = np.array([
    (25.3,32.1,38.8,25.4), 
    (25.3,29,31,21.1),
    (18.2,18.8,19.3,20.3),
    (18.3,24.3,15.7,24),
    (16.3,19,16.8,17.5)
]).T

In [32]:
np.median(xs) # Overall median
np.median(xs, axis = 0) # Column median
np.median(xs, axis = 1) # Row median

array([18.3, 24.3, 19.3, 21.1])

In [14]:
xs




array([[25.3, 25.3, 18.2, 18.3, 16.3],
       [32.1, 29. , 18.8, 24.3, 19. ],
       [38.8, 31. , 19.3, 15.7, 16.8],
       [25.4, 21.1, 20.3, 24. , 17.5]])

**3**. (50 points)

A Caesar cipher is a very simple method of encoding and decoding data. The cipher simply replaces characters with the character offset by $k$ places. For example, if the offset is 3, we replace `a` with `d`, `b` with `e` etc. The cipher wraps around so we replace `y` with `b`, `z` with `c` and so on. Punctuation, spaces and numbers are left unchanged.

- Write a function `encode` that takes as arguments a string and an integer offset and returns the encoded cipher.
- Write a function `decode` that takes as arguments a cipher and an integer offset and returns the decoded string. 
- Write a function `auto_decode` that takes as argument a cipher and uses a statistical method to guess the optimal offset to decode the cipher, assuming the original string is in English which has the following letter frequency:

```python
freq = {
 'a': 0.08167,
 'b': 0.01492,
 'c': 0.02782,
 'd': 0.04253,
 'e': 0.12702,
 'f': 0.02228,
 'g': 0.02015,
 'h': 0.06094,
 'i': 0.06966,
 'j': 0.00153,
 'k': 0.00772,
 'l': 0.04025,
 'm': 0.02406,
 'n': 0.06749,
 'o': 0.07507,
 'p': 0.01929,
 'q': 0.00095,
 'r': 0.05987,
 's': 0.06327,
 't': 0.09056,
 'u': 0.02758,
 'v': 0.00978,
 'w': 0.0236,
 'x': 0.0015,
 'y': 0.01974,
 'z': 0.00074
}
```

- Encode the following nursery rhyme using a random offset from 10 to 20, then recover the original using `auto_decode`:

```text
Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;
One for the master,
And one for the dame,
And one for the little boy
Who lives down the lane.
```